# ACCESS BARRA Grid Coordinates Extraction

## Overview

This notebook extracts grid cell coordinates from:
1. **ACCESS CM2 SSP245** (0.05° × 0.05° resolution) - AGDC grid
2. **BARRA SSP245** - BARRA grid system

It identifies grid cells with higher separation between the two systems and exports coordinates to CSV files.

## Input Directories

- **ACCESS files**: `C:\Users\ibian\Desktop\ClimAdapt\CMIP6\ACCESS CM2 SSP245\tasmax_ACCESS CM2 SSP245\`
- **BARRA files**: `C:\Users\ibian\Desktop\ClimAdapt\CMIP6\ACCESS CM2 SSP245\rsds_BARRA SSP245\`

## Output

Two CSV files saved to `C:\Users\ibian\Desktop\ClimAdapt\CMIP6\`:
- `AGDC_grid_coordinates.csv` - List of ACCESS (AGDC) grid coordinates
- `BARRA_grid_coordinates.csv` - List of BARRA grid coordinates with higher separation

## Section 1: Imports and Configuration

In [6]:
import pandas as pd
import numpy as np
import xarray as xr
import glob
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully")

Libraries imported successfully


## Section 2: Configuration

In [7]:
# Configuration
# ============================================================================

# Input directories
ACCESS_DIR = r"C:\Users\ibian\Desktop\ClimAdapt\CMIP6\ACCESS CM2 SSP245\tasmax_ACCESS CM2 SSP245"
BARRA_DIR = r"C:\Users\ibian\Desktop\ClimAdapt\CMIP6\ACCESS CM2 SSP245\rsds_BARRA SSP245"

# Output directory
OUTPUT_DIR = r"C:\Users\ibian\Desktop\ClimAdapt\CMIP6"

# Separation threshold (degrees) - coordinates with separation > this value will be included
SEPARATION_THRESHOLD = 0.025  # 0.025 degrees ≈ 2.5 km (half of 0.05° grid resolution)

# Western Australia boundaries (to filter coordinates during extraction)
WA_LAT_MIN = -35.0  # Southern boundary
WA_LAT_MAX = -13.5  # Northern boundary
WA_LON_MIN = 113.0  # Western boundary
WA_LON_MAX = 129.0  # Eastern boundary

print("="*70)
print("CONFIGURATION")
print("="*70)
print(f"ACCESS Directory: {ACCESS_DIR}")
print(f"BARRA Directory: {BARRA_DIR}")
print(f"Output Directory: {OUTPUT_DIR}")
print(f"Separation Threshold: {SEPARATION_THRESHOLD} degrees")
print(f"WA Boundaries: Lat {WA_LAT_MIN} to {WA_LAT_MAX}, Lon {WA_LON_MIN} to {WA_LON_MAX}")
print("="*70)

CONFIGURATION
ACCESS Directory: C:\Users\ibian\Desktop\ClimAdapt\CMIP6\ACCESS CM2 SSP245\tasmax_ACCESS CM2 SSP245
BARRA Directory: C:\Users\ibian\Desktop\ClimAdapt\CMIP6\ACCESS CM2 SSP245\rsds_BARRA SSP245
Output Directory: C:\Users\ibian\Desktop\ClimAdapt\CMIP6
Separation Threshold: 0.025 degrees
WA Boundaries: Lat -35.0 to -13.5, Lon 113.0 to 129.0


## Section 3: Extract ACCESS (AGDC) Grid Coordinates

In [8]:
def extract_access_grid_coordinates(access_dir):
    """
    Extract all grid cell coordinates from ACCESS CM2 NetCDF files.
    
    Parameters:
    -----------
    access_dir : str
        Directory containing ACCESS CM2 NetCDF files
    
    Returns:
    --------
    pd.DataFrame
        DataFrame with columns: latitude, longitude
    """
    print(f"\n{'='*70}")
    print("Extracting ACCESS (AGDC) Grid Coordinates")
    print(f"{'='*70}")
    
    # Find NetCDF files
    nc_files = sorted(glob.glob(os.path.join(access_dir, "*.nc")))
    
    if len(nc_files) == 0:
        print(f"  [ERROR] No NetCDF files found in {access_dir}")
        return None
    
    print(f"  Found {len(nc_files)} NetCDF files")
    
    # Use first file to extract grid coordinates
    try:
        ds = xr.open_dataset(nc_files[0], decode_times=False)
        
        # Find latitude and longitude coordinate names
        lat_name = None
        lon_name = None
        
        # Common coordinate names
        possible_lat_names = ['lat', 'latitude', 'y']
        possible_lon_names = ['lon', 'longitude', 'x']
        
        for coord_name in ds.coords:
            coord_lower = coord_name.lower()
            if coord_lower in possible_lat_names or 'lat' in coord_lower:
                lat_name = coord_name
            elif coord_lower in possible_lon_names or 'lon' in coord_lower:
                lon_name = coord_name
        
        if lat_name is None or lon_name is None:
            print(f"  [ERROR] Could not find latitude/longitude coordinates")
            print(f"  Available coordinates: {list(ds.coords.keys())}")
            ds.close()
            return None
        
        print(f"  Using coordinates: {lat_name}, {lon_name}")
        
        # Extract all latitude and longitude values
        lat_values = ds[lat_name].values
        lon_values = ds[lon_name].values
        
        # Create meshgrid of all coordinate combinations
        lon_grid, lat_grid = np.meshgrid(lon_values, lat_values)
        
        # Filter to Western Australia boundaries BEFORE creating DataFrame
        wa_mask = (
            (lat_grid >= WA_LAT_MIN) & (lat_grid <= WA_LAT_MAX) &
            (lon_grid >= WA_LON_MIN) & (lon_grid <= WA_LON_MAX)
        )
        
        # Flatten to get all coordinate pairs (only those within WA)
        coordinates = pd.DataFrame({
            'latitude': lat_grid[wa_mask],
            'longitude': lon_grid[wa_mask]
        })
        
        # Remove duplicates (in case of overlapping grids)
        coordinates = coordinates.drop_duplicates().reset_index(drop=True)
        
        # Sort by latitude, then longitude
        coordinates = coordinates.sort_values(['latitude', 'longitude']).reset_index(drop=True)
        
        print(f"  [OK] Extracted {len(coordinates)} unique grid coordinates")
        print(f"  Latitude range: {coordinates['latitude'].min():.4f} to {coordinates['latitude'].max():.4f}")
        print(f"  Longitude range: {coordinates['longitude'].min():.4f} to {coordinates['longitude'].max():.4f}")
        
        ds.close()
        
        return coordinates
        
    except Exception as e:
        print(f"  [ERROR] Failed to read NetCDF file: {e}")
        import traceback
        traceback.print_exc()
        return None

# Extract ACCESS coordinates
access_coords = extract_access_grid_coordinates(ACCESS_DIR)


Extracting ACCESS (AGDC) Grid Coordinates
  Found 30 NetCDF files
  Using coordinates: lat, lon
  [OK] Extracted 138351 unique grid coordinates
  Latitude range: -35.0000 to -13.5000
  Longitude range: 113.0000 to 129.0000


## Section 4: Extract BARRA Grid Coordinates

In [9]:
def extract_barra_grid_coordinates(barra_dir):
    """
    Extract all grid cell coordinates from BARRA NetCDF files.
    
    Parameters:
    -----------
    barra_dir : str
        Directory containing BARRA NetCDF files
    
    Returns:
    --------
    pd.DataFrame
        DataFrame with columns: latitude, longitude
    """
    print(f"\n{'='*70}")
    print("Extracting BARRA Grid Coordinates")
    print(f"{'='*70}")
    
    # Find NetCDF files
    nc_files = sorted(glob.glob(os.path.join(barra_dir, "*.nc")))
    
    if len(nc_files) == 0:
        print(f"  [ERROR] No NetCDF files found in {barra_dir}")
        return None
    
    print(f"  Found {len(nc_files)} NetCDF files")
    
    # Use first file to extract grid coordinates
    try:
        ds = xr.open_dataset(nc_files[0], decode_times=False)
        
        # Find latitude and longitude coordinate names
        lat_name = None
        lon_name = None
        
        # Common coordinate names
        possible_lat_names = ['lat', 'latitude', 'y']
        possible_lon_names = ['lon', 'longitude', 'x']
        
        for coord_name in ds.coords:
            coord_lower = coord_name.lower()
            if coord_lower in possible_lat_names or 'lat' in coord_lower:
                lat_name = coord_name
            elif coord_lower in possible_lon_names or 'lon' in coord_lower:
                lon_name = coord_name
        
        if lat_name is None or lon_name is None:
            print(f"  [ERROR] Could not find latitude/longitude coordinates")
            print(f"  Available coordinates: {list(ds.coords.keys())}")
            ds.close()
            return None
        
        print(f"  Using coordinates: {lat_name}, {lon_name}")
        
        # Extract all latitude and longitude values
        lat_values = ds[lat_name].values
        lon_values = ds[lon_name].values
        
        # Create meshgrid of all coordinate combinations
        lon_grid, lat_grid = np.meshgrid(lon_values, lat_values)
        
        # Filter to Western Australia boundaries BEFORE creating DataFrame
        wa_mask = (
            (lat_grid >= WA_LAT_MIN) & (lat_grid <= WA_LAT_MAX) &
            (lon_grid >= WA_LON_MIN) & (lon_grid <= WA_LON_MAX)
        )
        
        # Flatten to get all coordinate pairs (only those within WA)
        coordinates = pd.DataFrame({
            'latitude': lat_grid[wa_mask],
            'longitude': lon_grid[wa_mask]
        })
        
        # Remove duplicates
        coordinates = coordinates.drop_duplicates().reset_index(drop=True)
        
        # Sort by latitude, then longitude
        coordinates = coordinates.sort_values(['latitude', 'longitude']).reset_index(drop=True)
        
        print(f"  [OK] Extracted {len(coordinates)} unique grid coordinates")
        print(f"  Latitude range: {coordinates['latitude'].min():.4f} to {coordinates['latitude'].max():.4f}")
        print(f"  Longitude range: {coordinates['longitude'].min():.4f} to {coordinates['longitude'].max():.4f}")
        
        ds.close()
        
        return coordinates
        
    except Exception as e:
        print(f"  [ERROR] Failed to read NetCDF file: {e}")
        import traceback
        traceback.print_exc()
        return None

# Extract BARRA coordinates
barra_coords = extract_barra_grid_coordinates(BARRA_DIR)


Extracting BARRA Grid Coordinates
  Found 30 NetCDF files
  Using coordinates: lat, lon
  [OK] Extracted 138351 unique grid coordinates
  Latitude range: -35.0000 to -13.5000
  Longitude range: 113.0000 to 129.0000


In [10]:
# Grid Spacing Diagnostic Analysis
# ============================================================================
# This section verifies the actual grid spacing from the NetCDF files
# to confirm that ACCESS (5km) and BARRA (12km) have different resolutions

print(f"\n{'='*70}")
print("GRID SPACING DIAGNOSTIC ANALYSIS")
print(f"{'='*70}")

# Check ACCESS grid spacing
print(f"\n{'='*70}")
print("ACCESS Grid Spacing Analysis")
print(f"{'='*70}")
if access_coords is not None:
    # Get unique sorted lat/lon values
    access_lats = sorted(access_coords['latitude'].unique())
    access_lons = sorted(access_coords['longitude'].unique())
    
    # Calculate spacing
    if len(access_lats) > 1:
        lat_spacing = np.diff(access_lats)
        print(f"  Latitude spacing: min={lat_spacing.min():.6f}°, max={lat_spacing.max():.6f}°, mean={lat_spacing.mean():.6f}°")
        print(f"  Latitude spacing in km (approx): {lat_spacing.mean() * 111:.2f} km")
    if len(access_lons) > 1:
        lon_spacing = np.diff(access_lons)
        print(f"  Longitude spacing: min={lon_spacing.min():.6f}°, max={lon_spacing.max():.6f}°, mean={lon_spacing.mean():.6f}°")
        # Calculate longitude spacing in km (varies with latitude)
        mid_lat = access_lats[len(access_lats)//2]
        print(f"  Longitude spacing in km (approx at lat {mid_lat:.2f}°): {lon_spacing.mean() * 111 * np.cos(np.radians(mid_lat)):.2f} km")
    
    print(f"  Number of unique latitudes: {len(access_lats)}")
    print(f"  Number of unique longitudes: {len(access_lons)}")
    print(f"  Sample latitudes (first 5): {access_lats[:5]}")
    print(f"  Sample latitudes (last 5): {access_lats[-5:]}")
    print(f"  Sample longitudes (first 5): {access_lons[:5]}")
    print(f"  Sample longitudes (last 5): {access_lons[-5:]}")
else:
    print("  [ERROR] ACCESS coordinates not available")

# Check BARRA grid spacing
print(f"\n{'='*70}")
print("BARRA Grid Spacing Analysis")
print(f"{'='*70}")
if barra_coords is not None:
    # Get unique sorted lat/lon values
    barra_lats = sorted(barra_coords['latitude'].unique())
    barra_lons = sorted(barra_coords['longitude'].unique())
    
    # Calculate spacing
    if len(barra_lats) > 1:
        lat_spacing = np.diff(barra_lats)
        print(f"  Latitude spacing: min={lat_spacing.min():.6f}°, max={lat_spacing.max():.6f}°, mean={lat_spacing.mean():.6f}°")
        print(f"  Latitude spacing in km (approx): {lat_spacing.mean() * 111:.2f} km")
    if len(barra_lons) > 1:
        lon_spacing = np.diff(barra_lons)
        # Calculate longitude spacing in km (varies with latitude)
        mid_lat = barra_lats[len(barra_lats)//2]
        print(f"  Longitude spacing: min={lon_spacing.min():.6f}°, max={lon_spacing.max():.6f}°, mean={lon_spacing.mean():.6f}°")
        print(f"  Longitude spacing in km (approx at lat {mid_lat:.2f}°): {lon_spacing.mean() * 111 * np.cos(np.radians(mid_lat)):.2f} km")
    
    print(f"  Number of unique latitudes: {len(barra_lats)}")
    print(f"  Number of unique longitudes: {len(barra_lons)}")
    print(f"  Sample latitudes (first 5): {barra_lats[:5]}")
    print(f"  Sample latitudes (last 5): {barra_lats[-5:]}")
    print(f"  Sample longitudes (first 5): {barra_lons[:5]}")
    print(f"  Sample longitudes (last 5): {barra_lons[-5:]}")
else:
    print("  [ERROR] BARRA coordinates not available")

# Compare grids
print(f"\n{'='*70}")
print("Grid Comparison")
print(f"{'='*70}")
if access_coords is not None and barra_coords is not None:
    access_lats = sorted(access_coords['latitude'].unique())
    barra_lats = sorted(barra_coords['latitude'].unique())
    access_lons = sorted(access_coords['longitude'].unique())
    barra_lons = sorted(barra_coords['longitude'].unique())
    
    print(f"  ACCESS has {len(access_lats)} unique latitudes, BARRA has {len(barra_lats)}")
    print(f"  ACCESS has {len(access_lons)} unique longitudes, BARRA has {len(barra_lons)}")
    
    # Check if grids are identical
    if len(access_lats) == len(barra_lats) and len(access_lons) == len(barra_lons):
        lat_match = np.allclose(access_lats, barra_lats, atol=1e-6)
        lon_match = np.allclose(access_lons, barra_lons, atol=1e-6)
        if lat_match and lon_match:
            print(f"  [WARNING] Grids appear to be IDENTICAL! This suggests both files use the same grid.")
            print(f"  [WARNING] This would explain why separation is near zero.")
            print(f"  [WARNING] Please verify that the NetCDF files are from different grid systems.")
        else:
            print(f"  Grids have same dimensions but different coordinate values")
            print(f"  Latitude arrays match: {lat_match}")
            print(f"  Longitude arrays match: {lon_match}")
            if not lat_match:
                diff_idx = np.where(~np.isclose(access_lats, barra_lats, atol=1e-6))[0]
                if len(diff_idx) > 0:
                    print(f"  First differing lat index: {diff_idx[0]}")
            if not lon_match:
                diff_idx = np.where(~np.isclose(access_lons, barra_lons, atol=1e-6))[0]
                if len(diff_idx) > 0:
                    print(f"  First differing lon index: {diff_idx[0]}")
    else:
        print(f"  Grids have different dimensions - this is expected for different resolutions")
        print(f"  ACCESS grid is {'finer' if len(access_lats) > len(barra_lats) else 'coarser'} than BARRA grid")
        
    # Calculate expected spacing differences
    if len(access_lats) > 1 and len(barra_lats) > 1:
        access_lat_spacing = np.mean(np.diff(access_lats))
        barra_lat_spacing = np.mean(np.diff(barra_lats))
        print(f"\n  Expected separation based on grid spacing:")
        print(f"    ACCESS lat spacing: {access_lat_spacing:.6f}° ({access_lat_spacing * 111:.2f} km)")
        print(f"    BARRA lat spacing: {barra_lat_spacing:.6f}° ({barra_lat_spacing * 111:.2f} km)")
        print(f"    Spacing ratio: {barra_lat_spacing / access_lat_spacing:.2f}x")
        
print(f"\n{'='*70}")


GRID SPACING DIAGNOSTIC ANALYSIS

ACCESS Grid Spacing Analysis
  Latitude spacing: min=0.049999°, max=0.050003°, mean=0.050000°
  Latitude spacing in km (approx): 5.55 km
  Longitude spacing: min=0.049988°, max=0.050003°, mean=0.050000°
  Longitude spacing in km (approx at lat -24.25°): 5.06 km
  Number of unique latitudes: 431
  Number of unique longitudes: 321
  Sample latitudes (first 5): [np.float32(-35.0), np.float32(-34.95), np.float32(-34.9), np.float32(-34.85), np.float32(-34.8)]
  Sample latitudes (last 5): [np.float32(-13.7), np.float32(-13.65), np.float32(-13.6), np.float32(-13.55), np.float32(-13.5)]
  Sample longitudes (first 5): [np.float32(113.0), np.float32(113.05), np.float32(113.1), np.float32(113.15), np.float32(113.2)]
  Sample longitudes (last 5): [np.float32(128.8), np.float32(128.85), np.float32(128.9), np.float32(128.95), np.float32(129.0)]

BARRA Grid Spacing Analysis
  Latitude spacing: min=0.050000°, max=0.050000°, mean=0.050000°
  Latitude spacing in km (ap

## Section 5: Calculate Separation and Filter Coordinates

In [11]:
def calculate_separation(access_coords, barra_coords, threshold):
    """
    Calculate separation between ACCESS and BARRA grid coordinates.
    Find BARRA coordinates with higher separation from ACCESS grid.
    
    Parameters:
    -----------
    access_coords : pd.DataFrame
        ACCESS grid coordinates (columns: latitude, longitude)
    barra_coords : pd.DataFrame
        BARRA grid coordinates (columns: latitude, longitude)
    threshold : float
        Minimum separation threshold in degrees
    
    Returns:
    --------
    tuple: (access_coords_filtered, barra_coords_filtered, separation_info)
        Filtered coordinate DataFrames and separation information
    """
    print(f"\n{'='*70}")
    print("Calculating Grid Separation")
    print(f"{'='*70}")
    
    if access_coords is None or barra_coords is None:
        print("  [ERROR] Missing coordinate data")
        return None, None, None
    
    print(f"  ACCESS grid points: {len(access_coords)}")
    print(f"  BARRA grid points: {len(barra_coords)}")
    
    # Convert to numpy arrays for faster computation
    access_lat = access_coords['latitude'].values
    access_lon = access_coords['longitude'].values
    barra_lat = barra_coords['latitude'].values
    barra_lon = barra_coords['longitude'].values
    
    # Calculate minimum distance from each BARRA point to any ACCESS point
    print(f"  Calculating minimum distances...")
    min_distances = []
    
    # Use vectorized computation for efficiency
    for i, (b_lat, b_lon) in enumerate(zip(barra_lat, barra_lon)):
        # Calculate distance to all ACCESS points
        # Using Euclidean distance in lat/lon space
        lat_diff = access_lat - b_lat
        lon_diff = access_lon - b_lon
        distances = np.sqrt(lat_diff**2 + lon_diff**2)
        
        min_dist = np.min(distances)
        min_distances.append(min_dist)
        
        if (i + 1) % 1000 == 0:
            print(f"    Processed {i + 1}/{len(barra_lat)} points...")
    
    # Add separation column to BARRA coordinates
    barra_coords_with_sep = barra_coords.copy()
    barra_coords_with_sep['separation_degrees'] = min_distances
    
    # Filter BARRA coordinates with separation > threshold
    barra_filtered = barra_coords_with_sep[
        barra_coords_with_sep['separation_degrees'] > threshold
    ].copy()
    
    print(f"\n  Separation Statistics:")
    print(f"    Minimum separation: {np.min(min_distances):.6f} degrees")
    print(f"    Maximum separation: {np.max(min_distances):.6f} degrees")
    print(f"    Mean separation: {np.mean(min_distances):.6f} degrees")
    print(f"    Median separation: {np.median(min_distances):.6f} degrees")
    
    print(f"\n  Filtering Results:")
    print(f"    Threshold: {threshold} degrees")
    print(f"    BARRA points with separation > threshold: {len(barra_filtered)} / {len(barra_coords)}")
    print(f"    Percentage: {100 * len(barra_filtered) / len(barra_coords):.2f}%")
    
    # Sort by separation (descending)
    barra_filtered = barra_filtered.sort_values('separation_degrees', ascending=False).reset_index(drop=True)
    
    return access_coords, barra_filtered, barra_coords_with_sep

# Calculate separation
access_final, barra_filtered, barra_with_sep = calculate_separation(
    access_coords, barra_coords, SEPARATION_THRESHOLD
)


Calculating Grid Separation
  ACCESS grid points: 138351
  BARRA grid points: 138351
  Calculating minimum distances...
    Processed 1000/138351 points...
    Processed 2000/138351 points...
    Processed 3000/138351 points...
    Processed 4000/138351 points...
    Processed 5000/138351 points...
    Processed 6000/138351 points...
    Processed 7000/138351 points...
    Processed 8000/138351 points...
    Processed 9000/138351 points...
    Processed 10000/138351 points...
    Processed 11000/138351 points...
    Processed 12000/138351 points...
    Processed 13000/138351 points...
    Processed 14000/138351 points...
    Processed 15000/138351 points...
    Processed 16000/138351 points...
    Processed 17000/138351 points...
    Processed 18000/138351 points...
    Processed 19000/138351 points...
    Processed 20000/138351 points...


KeyboardInterrupt: 

## Section 6: Save Coordinates to CSV Files

In [ ]:
# Save coordinates to CSV files
print(f"\n{'='*70}")
print("Saving Coordinates to CSV Files")
print(f"{'='*70}")

# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Save ACCESS (AGDC) coordinates
if access_final is not None:
    access_output_path = os.path.join(OUTPUT_DIR, "AGDC_grid_coordinates.csv")
    access_final.to_csv(access_output_path, index=False, encoding='utf-8')
    print(f"\n  [OK] Saved ACCESS (AGDC) coordinates: {access_output_path}")
    print(f"       Total coordinates: {len(access_final)}")
else:
    print(f"\n  [ERROR] No ACCESS coordinates to save")

# Save BARRA coordinates (with higher separation)
if barra_filtered is not None and len(barra_filtered) > 0:
    barra_output_path = os.path.join(OUTPUT_DIR, "BARRA_grid_coordinates.csv")
    # Save only latitude and longitude (drop separation column for cleaner output)
    barra_filtered[['latitude', 'longitude']].to_csv(barra_output_path, index=False, encoding='utf-8')
    print(f"\n  [OK] Saved BARRA coordinates: {barra_output_path}")
    print(f"       Total coordinates: {len(barra_filtered)}")
    print(f"       Separation range: {barra_filtered['separation_degrees'].min():.6f} to {barra_filtered['separation_degrees'].max():.6f} degrees")
    
    # Also save version with separation column for reference
    barra_detailed_path = os.path.join(OUTPUT_DIR, "BARRA_grid_coordinates_with_separation.csv")
    barra_filtered.to_csv(barra_detailed_path, index=False, encoding='utf-8')
    print(f"       Detailed version (with separation): {barra_detailed_path}")
else:
    print(f"\n  [WARNING] No BARRA coordinates with separation > {SEPARATION_THRESHOLD} degrees")
    if barra_with_sep is not None:
        # Save all BARRA coordinates with separation info
        barra_output_path = os.path.join(OUTPUT_DIR, "BARRA_grid_coordinates_all.csv")
        barra_with_sep.to_csv(barra_output_path, index=False, encoding='utf-8')
        print(f"       Saved all BARRA coordinates with separation info: {barra_output_path}")

print(f"\n{'='*70}")
print("[COMPLETE] Coordinate extraction completed!")
print(f"{'='*70}")


Saving Coordinates to CSV Files

  [OK] Saved ACCESS (AGDC) coordinates: C:\Users\ibian\Desktop\ClimAdapt\CMIP6\AGDC_grid_coordinates.csv
       Total coordinates: 138351

  [WARNING] No BARRA coordinates with separation > 0.025 degrees
       Saved all BARRA coordinates with separation info: C:\Users\ibian\Desktop\ClimAdapt\CMIP6\BARRA_grid_coordinates_all.csv

[COMPLETE] Coordinate extraction completed!
